# Pytorch Basic

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

### torch.tensor 함수

`tensor(data, dtype=None, device=None, requires_grad=False) -> Tensor`

     data (array_like): list, tuple, numpy ndarray, scalar, and other types.
     dtype :  `None`인 경우 `data`에서 데이터 유형을 유추합니다.  
     device : `cpu`, `cuda`  
     require_grad(bool, optional): autograd가 작업을 기록해야 하는 경우

Float torch tensor 생성

In [2]:
# numpy 생성
a = np.ones((2, 3), dtype="float32")
# tensor 생성
b = torch.tensor(a)
# dtype
print(a.dtype, '\t', b.dtype)

float32 	 torch.float32


Integer tensor 생성

In [3]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.int32)
x

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)

### Tensor 생성

- 무작위로 초기화된 행렬 생성 (uniform distribution)

In [4]:
x = torch.rand(5, 3)
print(x)

tensor([[0.2858, 0.1407, 0.3070],
        [0.7996, 0.0583, 0.8411],
        [0.8182, 0.7560, 0.7350],
        [0.0206, 0.2115, 0.6574],
        [0.2271, 0.3695, 0.6418]])


## Tensor 의 shape & dimension (rank)

In [5]:
a = torch.Tensor([0, 1, 2, 3, 4])
a.size()

torch.Size([5])

In [6]:
# size() 와 shape 은 alias
a.shape

torch.Size([5])

In [7]:
a.ndimension()

1

## numpy 와 tensor 간의 호환성

- memory 를 공유하므로 하나를 수정하면 나머지에 모두 반영  

In [8]:
numpy_array = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])       # numpy array
torch_tensor = torch.from_numpy(numpy_array)               # torch tensor
torch_tensor

tensor([0., 1., 2., 3., 4., 5.], dtype=torch.float64)

In [9]:
back_to_numpy = torch_tensor.numpy()           # numpy array
back_to_numpy

array([0., 1., 2., 3., 4., 5.])

In [10]:
numpy_array, torch_tensor, back_to_numpy

(array([0., 1., 2., 3., 4., 5.]),
 tensor([0., 1., 2., 3., 4., 5.], dtype=torch.float64),
 array([0., 1., 2., 3., 4., 5.]))

In [11]:
back_to_numpy[0] = 100

numpy_array, torch_tensor, back_to_numpy

(array([100.,   1.,   2.,   3.,   4.,   5.]),
 tensor([100.,   1.,   2.,   3.,   4.,   5.], dtype=torch.float64),
 array([100.,   1.,   2.,   3.,   4.,   5.]))

## Scalar value

- 만약 tensor에 하나의 값만 존재한다면, ``.item()`` method를 사용하여 숫자 값을 얻을 수 있습니다.  

In [12]:
x = torch.randn(1)
x.item()

-0.19019469618797302

In [13]:
a = torch.tensor([5., 3., 4., 1.])

print(a[0])
print(a[0].item())

tensor(5.)
5.0


- tensor 가 array 형태인 경우 `numpy()` method 를 통해 ndarray 반환

In [14]:
x = torch.randn(2)
print(x)
print()
print(x.numpy())

tensor([0.4472, 0.6132])

[0.44719145 0.61316615]


## Basic Operations - numpy 와 동일

In [15]:
# element-wise 덧셈, 뺄셈
u = torch.tensor([1.0, 0.0])
v = torch.tensor([0.0, 1.0])
z = u + v
z

tensor([1., 1.])

In [16]:
z = u - v
z

tensor([ 1., -1.])

In [17]:
# element-wise 곱셈
u = torch.tensor([1, 2])
v = torch.tensor([3, 2])
z = u * v
z

tensor([3, 4])

In [18]:
# 스칼라 곱
y = torch.tensor([1, 2])
z = 2 * y
z

tensor([2, 4])

In [19]:
# dot product
u = torch.tensor([1, 2])
v = torch.tensor([3, 1])
result = torch.dot(u, v)
result

tensor(5)

## TORCH.AUTOGRAD를 이용한 자동 미분

- autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공   

- 신경망을 훈련할 때 가장 자주 사용되는 알고리즘은 역전파이다. 이 알고리즘에서 매개 변수 (모델 가중치)는 주어진 매개 변수에 대한 손실 함수의 기울기에 따라 조정된다.  

- 이러한 그래디언트를 계산하기 위해 PyTorch에는 torch.autograd라는 내장 미분 엔진이 있다.

- ``.requires_grad`` 속성을 True 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작  

- 계산이 완료된 후 ``.backward()`` 를 호출하여 모든 변화도(gradient)를 자동으로 계산  

- 이 Tensor의 변화도는 ``.grad`` 속성에 누적  

- Tensor가 기록을 추적하는 것을 중단하게 하려면, ``.detach()`` 를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지한다.

- 도함수를 계산하기 위해서는 Tensor 의 ``.backward()`` 를 호출

In [20]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [21]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

y 를 x 에 대하여 미분  
$$y(x)=x^2$$

$$\frac{dy(x=2)}{dx}=2x=4 \rightarrow {y.backward()}$$  

In [22]:
y.backward()

x.grad

tensor(4.)

## 편미분
$$f(u, v) = uv + u^2$$

$$\frac{\partial{f(u, v)}}{\partial {u}}=v+2u$$
$$\frac{\partial{f(u, v)}}{\partial {v}}=u$$

In [23]:
u = torch.tensor(1.0, requires_grad=True)
v = torch.tensor(2.0, requires_grad=True)

f = u * v + u ** 2
f

tensor(3., grad_fn=<AddBackward0>)

In [24]:
f.backward()

In [25]:
print(u.grad)
print(v.grad)

tensor(4.)
tensor(1.)


## 연습 문제

1. Tensor 생성  
3x3 크기의 무작위로 초기화된 텐서를 생성하세요.  
이 텐서의 데이터 타입을 확인하세요.  

힌트: torch.rand()와 .dtype를 활용하세요.  
예상 출력: (3, 3) 모양의 텐서와 데이터 타입 출력.

2. Tensor의 Shape와 Dimension  
[1.0, 2.0, 3.0, 4.0, 5.0] 값을 가진 1D 텐서를 생성하세요.  
이 텐서의 크기(size)와 차원(ndimension)을 확인하세요.

힌트: torch.Tensor(), .size(), .ndimension()을 활용하세요.  
예상 출력: 텐서의 크기 (5,)와 차원 1.

3. Numpy와 Tensor 간의 호환성  
NumPy 배열 [1.0, 2.0, 3.0]을 생성하세요.  
이 배열을 PyTorch 텐서로 변환하고, 다시 NumPy 배열로 변환하세요.  
NumPy 배열을 수정한 후, PyTorch 텐서 값이 어떻게 변하는지 확인하세요.

힌트: torch.from_numpy()와 .numpy()를 활용하세요.  
예상 출력: NumPy 배열과 텐서 값이 동기화되어 변경됨.

4. Basic Operations  
[1, 2, 3]과 [4, 5, 6]으로 구성된 두 개의 텐서를 생성하세요.  
다음 연산을 수행하세요:  
두 텐서의 합.  
두 텐서의 차.  
두 텐서의 원소별 곱.  
첫 번째 텐서에 3을 곱한 결과.

힌트: +, -, * 연산과 스칼라 곱셈을 활용하세요.  
예상 출력: 합, 차, 원소별 곱, 스칼라 곱 결과.